In [16]:
# Step 1: Imports
from dataclasses import dataclass
from pathlib import Path
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator 
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
from cnnClassifier import logger


In [17]:
from cnnClassifier.constants import *
from Brain_tumor_classifier.utils.common import read_yaml, create_directories

In [18]:
# Step 2: Entity

from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataTransformationConfig:
    root_dir: Path
    train_data_dir: Path
    test_data_dir: Path
    img_height: int
    img_width: int
    batch_size: int
    aug_params: dict


In [20]:
# Step 3: Configuration Manager
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH
    ):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation
        params = self.params

        create_directories([config.root_dir])

        return DataTransformationConfig(
            root_dir = config.root_dir,
            train_data_dir = config.train_data_dir,
            test_data_dir = config.test_data_dir,
            img_height = params.IMAGE_SIZE,
            img_width = params.IMAGE_SIZE,
            batch_size = params.BATCH_SIZE,
            aug_params = params.AUGMENTATION
        )


In [21]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [22]:
class DataTransformation:
    def __init__(self, config: DataTransformationConfig):
        self.config = config

    def get_data_generators(self):
        train_datagen = ImageDataGenerator(
            rescale=1./255,
            rotation_range=self.config.aug_params['rotation_range'],
            width_shift_range=self.config.aug_params['width_shift_range'],
            height_shift_range=self.config.aug_params['height_shift_range'],
            shear_range=self.config.aug_params['shear_range'],
            zoom_range=self.config.aug_params['zoom_range'],
            horizontal_flip=self.config.aug_params['horizontal_flip'],
            fill_mode='nearest'
        )

        test_datagen = ImageDataGenerator(rescale=1./255)

        train_generator = train_datagen.flow_from_directory(
            directory=str(self.config.train_data_dir),
            target_size=(self.config.img_height, self.config.img_width),
            batch_size=self.config.batch_size,
            class_mode='categorical'
        )

        test_generator = test_datagen.flow_from_directory(
            directory=str(self.config.test_data_dir),
            target_size=(self.config.img_height, self.config.img_width),
            batch_size=self.config.batch_size,
            class_mode='categorical'
        )

        return train_generator, test_generator


In [11]:
import os
print(os.getcwd())


c:\Users\devan\Downloads\Brain-Tumor-\research


In [12]:
os.chdir("../")

In [23]:
# Step 5: Run pipeline in notebook
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTransformation(config=data_transformation_config)
    train_gen, test_gen = data_transformation.get_data_generators()
    logger.info("Data Transformation completed successfully.")
except Exception as e:
    logger.exception(e)
    raise e


[2025-08-15 17:15:47,863: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-08-15 17:15:47,865: INFO: common: yaml file: params.yaml loaded successfully]
[2025-08-15 17:15:47,867: INFO: common: created directory at: artifacts]
[2025-08-15 17:15:47,868: INFO: common: created directory at: artifacts/data_transformation]
Found 202 images belonging to 2 classes.
Found 51 images belonging to 2 classes.
[2025-08-15 17:15:47,900: INFO: 4222976568: Data Transformation completed successfully.]
